In [1]:
from math import lgamma
from numba import jit
import numpy as np
from scipy.stats import beta

# Описание работы Байесовских алгоритмов

## Принцип работы


## Основные метрики

### Вероятность превосходства для конверсии

В случае алгоритма на конверсию у нас в качестве априорного распределения берется
биномиальное. Для того чтобы пересчитать апостериорную вероятность для биномиального
распределения, используется сопряженное ему бета-распределение.
Его можно посчитать двумя способами: по точной формуле или при помощи генерации
случайных величин из бета-распределения с параметрами на момент пересчета.

In [10]:
# Случай точной формулы
@jit
def h(a, b, c, d):
    num = lgamma(a + c) + lgamma(b + d) + lgamma(a + b) + lgamma(c + d)
    den = lgamma(a) + lgamma(b) + lgamma(c) + lgamma(d) + lgamma(a + b + c + d)
    return np.exp(num - den)


@jit
def g0(a, b, c):
    return np.exp(lgamma(a + b) + lgamma(a + c) - (lgamma(a + b + c) + lgamma(a)))


@jit
def hiter(a, b, c, d):
    while d > 1:
        d -= 1
        yield h(a, b, c, d) / d


def g(a, b, c, d):
    return g0(a, b, c) + sum(hiter(a, b, c, d))


# Финальная функция для точной формулы
def probability_superiority_integrity(alphas, betas):
    return g(alphas[0], betas[0], alphas[1], betas[1])


# Случай сэмплирования
def probability_superiority_sampled(alphas, betas, size=1000000):
    """
    Calculate expected losses for beta distribution
    :param size: number of random values
    :param alphas: alpha params
    :param betas: beta params
    :return:
    """
    control_thetas = beta.rvs(alphas[0], betas[0], size=size)
    test_thetas = beta.rvs(alphas[1], betas[1], size=size)
    difference = test_thetas < control_thetas
    prob_super = np.sum(difference) / size  # probability superiority for
    return prob_super

# Посмотрим на результаты, полученные разными способами
alphas, betas = [10, 20], [100, 200]
print(f"Точная формула: {probability_superiority_integrity(alphas, betas)}")
print(f"Сэмплирование: {probability_superiority_sampled(alphas, betas)}")

Точная формула: 0.48516728652925084
Сэмплирование: 0.485432


Отметим, что сэмплирование приводит примерно к тому же результату, но при большом
количестве генераций случайных распределений (например, 1 млн).


### Ожидаемые потери
